In [1]:
import xgboost
import numpy as np
import shap
import time

In [2]:
from iml.common import convert_to_instance, convert_to_model, match_instance_to_data, match_model_to_data, convert_to_instance_with_index
from iml.explanations import AdditiveExplanation
from iml.links import convert_to_link, IdentityLink
from iml.datatypes import convert_to_data, DenseData
import logging
from iml.explanations import AdditiveExplanation

log = logging.getLogger('shap')
from shap import KernelExplainer
class IMEExplainer(KernelExplainer):
    """ This is an implementation of the IME explanation method (aka. Shapley sampling values)
    
    IME was proposed in "An Efficient Explanation of Individual Classifications using Game Theory",
    Erik Štrumbelj, Igor Kononenko, JMLR 2010
    """
    
    def __init__(self, model, data, **kwargs):
        # silence warning about large datasets
        level = log.level
        log.setLevel(logging.ERROR)
        super(IMEExplainer, self).__init__(model, data, **kwargs)
        log.setLevel(level)
    
    def explain(self, incoming_instance, **kwargs):
        # convert incoming input to a standardized iml object
        instance = convert_to_instance(incoming_instance)
        match_instance_to_data(instance, self.data)
        
        # pick a reasonable number of samples if the user didn't specify how many they wanted
        self.nsamples = kwargs.get("nsamples", 0)
        if self.nsamples == 0:
            self.nsamples = 1000 * self.P
        
        # divide up the samples among the features
        self.nsamples_each = np.ones(self.P, dtype=np.int64) * 2 * (self.nsamples // (self.P * 2))
        for i in range((self.nsamples % (self.P * 2)) // 2):
            self.nsamples_each[i] += 2
        
        model_out = self.model.f(instance.x)
        
        # explain every feature
        phi = np.zeros(self.P)
        self.X_masked = np.zeros((self.nsamples_each.max(), X.shape[1]))
        for i in range(self.P):
            phi[i] = self.ime(i, self.model.f, instance.x, self.data.data, nsamples=self.nsamples_each[i])
        phi = np.array(phi)
        
        return AdditiveExplanation(self.link.f(1), self.link.f(1), phi, np.zeros(len(phi)), instance, self.link,
                                   self.model, self.data)
        
        
    def ime(self, j, f, x, X, nsamples=10):
        assert nsamples % 2 == 0, "nsamples must be divisible by 2!"
        X_masked = self.X_masked[:nsamples,:]
        inds = np.arange(X.shape[1])

        for i in range(0, nsamples//2):
            np.random.shuffle(inds)
            pos = np.where(inds == j)[0][0]
            rind = np.random.randint(X.shape[0])
            X_masked[i,:] = x
            X_masked[i,inds[pos+1:]] = X[rind,inds[pos+1:]]
            X_masked[-(i+1),:] = x
            X_masked[-(i+1),inds[pos:]] = X[rind,inds[pos:]]
            
        evals = f(X_masked)
        
        evals_on = evals[:nsamples//2]
        evals_off = evals[nsamples//2:][::-1]
        
        return np.mean(evals[:nsamples//2] - evals[nsamples//2:])

In [94]:
from tqdm import tqdm

tree_shap_times = []
kernel_shap_times = []
ime_times = []
nreps = 10

N = 1000
X_full = np.random.randn(N, 20)
y = np.random.randn(N)

for M in range(4,8):
    ts = []
    tree_shap_time = 0
    kernel_shap_time = 0
    ime_time = 0
    for k in tqdm(range(nreps)):
#         print()
         #+ ((X > 0).sum(1) % 2)
        X = X_full[:,:M]

        model = xgboost.train({"eta": 1}, xgboost.DMatrix(X, y), 1000)

        def f(x):
            return model.predict(xgboost.DMatrix(x))


        start = time.time()
        shap_values = shap.TreeExplainer(model).shap_values(X)
        tree_shap_time += time.time() - start
#         print("Tree SHAP:", tree_shap_time, "seconds")

        shap_stddev = shap_values.std(0)[:-1].mean()

#         print("mean std dev of SHAP values over samples:", shap_stddev)

        e = shap.KernelExplainer(f, X.mean(0).reshape(1,M))
        nsamples = 200
#         print(shap_stddev/20)
        for j in range(2000):
            #print(nsamples)
            start = time.time()
            std_dev = np.vstack([e.shap_values(X[:1,:], silent=True, nsamples=nsamples) for i in range(50)]).std(0)[:-1].mean()
            iter_time = (time.time() - start)/50
            #print(std_dev)
            if std_dev < shap_stddev/20:
#                 print("KernelExplainer", nsamples)
#                 print("KernelExplainer", std_dev)
#                 print("KernelExplainer", iter_time, "seconds")
                kernel_shap_time += iter_time * 1000
                break
            nsamples += int(nsamples * 0.5)

        e = IMEExplainer(f, X.mean(0).reshape(1,M))
        nsamples = 200
        for j in range(2000):
        #     print()
        #     print(nsamples)
            start = time.time()
            std_dev = np.vstack([e.shap_values(X[:1,:], silent=True, nsamples=nsamples) for i in range(50)]).std(0)[:-1].mean()
        #     print("time", (time.time() - start)/50)
        #     print(std_dev)
            iter_time = (time.time() - start)/50
            if std_dev < shap_stddev/20:
#                 print("IMEExplainer", nsamples)
#                 print("IMEExplainer", std_dev)
#                 print("IMEExplainer", iter_time, "seconds")
                ime_time += iter_time * 1000
                break
            nsamples += int(nsamples * 0.5)

    tree_shap_times.append(tree_shap_time / nreps)
    kernel_shap_times.append(kernel_shap_time / nreps)
    ime_times.append(ime_time / nreps)
    print("TreeExplainer", tree_shap_times[-1])
    print("KernelExplainer", kernel_shap_times[-1])
    print("IMEExplainer", ime_times[-1])


  0%|          | 0/10 [00:00<?, ?it/s]

TreeExplainer 0.5667633771896362
KernelExplainer 1.6998610496520996
IMEExplainer 36.06944036483765


  0%|          | 0/10 [00:00<?, ?it/s]

TreeExplainer 0.5962720394134522
KernelExplainer 2.410202980041504
IMEExplainer 24.168269634246826


  0%|          | 0/10 [00:00<?, ?it/s]

TreeExplainer 0.5785245656967163
KernelExplainer 3.6281538009643555
IMEExplainer 170.16926431655884


100%|██████████| 10/10 [06:38<00:00, 41.04s/it]

TreeExplainer 0.5870000839233398
KernelExplainer 5.659902095794678
IMEExplainer 254.79452514648438


In [96]:
model.predict(xgboost.DMatrix(X)).mean()

-0.035476692

In [97]:
shap.TreeExplainer(model).shap_values(X)

array([[ 0.35859036,  0.02820558, -0.0797407 , ...,  0.257011  ,
         0.14601958, -0.03547668],
       [-0.13089252,  0.05794828, -0.25855556, ...,  0.13599055,
         0.05735664, -0.03547668],
       [-0.06776153,  0.10805923, -0.15713027, ..., -0.03505304,
        -0.126518  , -0.03547668],
       ...,
       [ 0.2136814 ,  0.26080823, -0.28865245, ...,  0.37536186,
         0.0192902 , -0.03547668],
       [-0.26988652, -0.4866938 ,  0.08402091, ..., -0.9819344 ,
         0.10559861, -0.03547668],
       [ 0.2289274 ,  0.34278524,  0.04976799, ...,  0.17089835,
        -0.2589223 , -0.03547668]], dtype=float32)

In [68]:
e = shap.KernelExplainer(f, X.mean(0).reshape(1,M))
np.vstack([e.shap_values(X[:1,:], silent=True, nsamples=100) for i in range(50)]).std(0)[:-1].mean()

0.03536954622922005

In [72]:
e = shap.KernelExplainer(f, X.mean(0).reshape(1,M))
nsamples = 200
print(shap_stddev/20)
for j in range(2000):
    print(nsamples)
    start = time.time()
    std_dev = np.vstack([e.shap_values(X[:1,:], silent=True, nsamples=nsamples) for i in range(50)]).std(0)[:-1].mean()
    iter_time = time.time() - start)/50
    print(std_dev)
    if std_dev < shap_stddev/20:
        print(nsamples)
        break
    nsamples += int(nsamples * 0.2)

0.01028014048933983
200
time 0.017876300811767578
0.030893178278808088
240
time 0.01951019763946533
0.031826491957925064
288
time 0.020209121704101562
0.02404333546749491
345
time 0.026800222396850586
0.02191601539734573
414
time 0.03098787784576416
0.019460386139078127
496
time 0.035751018524169925
0.016387066172673846
595
time 0.04151914119720459
0.014285933327475487
714
time 0.04223478317260742
0.007730075891320169
714


In [74]:
e = IMEExplainer(f, X.mean(0).reshape(1,M))
nsamples = 200
print(shap_stddev/20)
for j in range(2000):
    print()
    print(nsamples)
    start = time.time()
    std_dev = np.vstack([e.shap_values(X[:1,:], silent=True, nsamples=nsamples) for i in range(50)]).std(0)[:-1].mean()
    print("time", (time.time() - start)/50)
    print(std_dev)
    if std_dev < shap_stddev/20:
        print(nsamples)
        break
    nsamples += int(nsamples * 0.2)

0.01028014048933983
200
time 0.00873462200164795
0.11283049739292224
240
time 0.008001022338867188
0.11367197853719405
288
time 0.011501140594482422
0.1015653587243526
345
time 0.009341177940368652
0.09609185923926047
414
time 0.011126718521118163
0.09085558541161791
496
time 0.013986682891845703
0.07316596561338537
595
time 0.015461082458496095
0.07250843647631118
714
time 0.01717233657836914
0.06397322007857624
856
time 0.022870540618896484
0.05976986733191383
1027
time 0.023901219367980956
0.05245597953754562
1232
time 0.027827000617980956
0.04745224231486926
1478
time 0.03509308338165283
0.044717441080350424
1773
time 0.03991847991943359
0.04005476391599552
2127
time 0.049641480445861814
0.03812949961499427
2552
time 0.055928120613098146
0.035810585105934746
3062
time 0.06629895687103271
0.03023017573129038
3674
time 0.08427309989929199
0.028151972927120038
4408
time 0.09465731620788574
0.0269187187096272
5289
time 0.11998224258422852
0.022239860334659148
6346
time 0.14299814224243

In [75]:
0.56939 * 1000

569.39

In [36]:
np.std([IMEExplainer(f, X.mean(0).reshape(1,M)).shap_values(X[:1,:], silent=True, nsamples=1000)[0,0] for i in range(10)])

0.10860389655048514

In [20]:
[shap.KernelExplainer(f, X.mean(0).reshape(1,M)).shap_values(X[:1,:], silent=True, nsamples=1000)[0,0] for i in range(100)]

[-0.023676379073087367,
 -0.016465743042426484,
 -0.01704981192550009,
 -0.018463699458183006,
 -0.015413553059016158,
 -0.018366587057752404,
 -0.018345955714953288,
 -0.01926604949186531,
 -0.020396613851721868,
 -0.022111201288591047,
 -0.021292876352076523,
 -0.016925113704097466,
 -0.02094528676946239,
 -0.017590427283719918,
 -0.019455733580809376,
 -0.01867957831829911,
 -0.01949199924313576,
 -0.02101184177451282,
 -0.02017175661418666,
 -0.02096841698600091,
 -0.018895762603090914,
 -0.019425187391885285,
 -0.018764209898906616,
 -0.02034981563789895,
 -0.016667910548829656,
 -0.019427677621697337,
 -0.019561391198267386,
 -0.02373383328585584,
 -0.017769291765458606,
 -0.014731135458020739,
 -0.01941898747203586,
 -0.020089057050320602,
 -0.019028685125618505,
 -0.01962223838514693,
 -0.014568940935971698,
 -0.01879752859462427,
 -0.018104240312813935,
 -0.019211399989558977,
 -0.021056719830586573,
 -0.018778172837663646,
 -0.017658694005700082,
 -0.019683240102995762,
 -0.0162219930410617,
 -0.019259564972963772,
 -0.01949005847120361,
 -0.020469469407407792,
 -0.012816015402021158,
 -0.017980626203507558,
 -0.020897377427076758,
 -0.01868522102558161,
 -0.019841164347362744,
 -0.01599499867003788,
 -0.023435164030154998,
 -0.021409903581716072,
 -0.021450264083101744,
 -0.019248117394281006,
 -0.019173267845621733,
 -0.021201344451321935,
 -0.013850440640304496,
 -0.021287986511504587,
 -0.017783326099606533,
 -0.019453412161091754,
 -0.01843054464371413,
 -0.017254967740485454,
 -0.019103215848530974,
 -0.01879518729784055,
 -0.016519074933045257,
 -0.020527604263832898,
 -0.014484539065507975,
 -0.019919451452777043,
 -0.01791397307200243,
 -0.014867680956663631,
 -0.018727430647644053,
 -0.018632469437516508,
 -0.018405491524922116,
 -0.01747364031491877,
 -0.01699568669949486,
 -0.01874634675402474,
 -0.01885201916934609,
 -0.02082471371013129,
 -0.018097378494376464,
 -0.020461013612397938,
 -0.01658913152329026,
 -0.021541127531476613,
 -0.021776620067082225,
 -0.020021620037416813,
 -0.01784317502978247,
 -0.02140662002036492,
 -0.02027189405855656,
 -0.021590607084915926,
 -0.018937654008019444,
 -0.01834817726407234,
 -0.01792343162344523,
 -0.02115901867004244,
 -0.019107028260820377,
 -0.02001855177068948,
 -0.019976099903129818,
 -0.013791710928271679,
 -0.021506799835474072,
 -0.01541739020484878]

In [7]:
def f(x):
    return model.predict(xgboost.DMatrix(x))

start = time.time()
shap_values2 = shap.KernelExplainer(f, X.mean(0).reshape(1,M)).shap_values(X)
print(time.time() - start)

  7%|▋         | 74/1000 [00:04<00:50, 18.27it/s]


KeyboardInterrupt: 

In [22]:
start = time.time()
IMEExplainer(f, X.mean(0).reshape(1,M)).shap_values(X)
print(time.time() - start)

  4%|▍         | 41/1000 [00:13<05:22,  2.97it/s]

KeyboardInterrupt: 